In [1]:
import autograd
import autograd.numpy as np


# Paragami debugging

In [19]:
import paragami
import copy
import unittest
from numpy.testing import assert_array_almost_equal
import scipy as sp
import scipy as osp

import itertools
import json
import collections
import time

#  Profiling

In [127]:
class DummyTest(unittest.TestCase):
    pass
testcase = DummyTest()
self = DummyTest()
check_equal = assert_array_almost_equal

# array_pattern = paragami.NumericArrayPattern(
#     shape=(4, ), lb=-1, ub=10.0)
# pattern = paragami.PatternArray((2, 3), array_pattern)

pattern = paragami.PSDSymmetricMatrixPattern(100)

#pattern = paragami.NumericArrayPattern(shape=(500, 10, 10), lb=-1, ub=10.0)



In [137]:
import cProfile
import pstats
valid_value = pattern.random()

# Execute required methods.
empty_val = pattern.empty(valid=True)
flat_vale = pattern.flatten(empty_val, free=True)
profile = True

if profile:
    pr = cProfile.Profile()
    pr.enable()

tic = time.time()
for _ in range(500):
    pattern.flatten(empty_val, free=True, validate_value=False)
tic = time.time() - tic; print(tic)

if profile:
    pr.disable()

0.3919086456298828


In [138]:
ps = pstats.Stats(pr).strip_dirs().sort_stats('cumulative')
ps.print_stats()

         75152 function calls (74152 primitive calls) in 0.393 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    0.392    0.078 interactiveshell.py:3281(run_code)
        5    0.000    0.000    0.392    0.078 {built-in method builtins.exec}
        1    0.001    0.001    0.392    0.392 <ipython-input-137-23e209e8308e>:15(<module>)
      500    0.002    0.000    0.390    0.001 psdmatrix_patterns.py:291(flatten)
      500    0.011    0.000    0.387    0.001 psdmatrix_patterns.py:141(_pack_posdef_matrix)
     4500    0.011    0.000    0.309    0.000 tracer.py:35(f_wrapped)
5500/4500    0.007    0.000    0.232    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      500    0.046    0.000    0.143    0.000 psdmatrix_patterns.py:35(_vectorize_ld_matrix)
      500    0.001    0.000    0.139    0.000 <__array_function__ internals>:2(cholesky)
      500    0.120    0.000  

In [ ]:
empty_val = pattern.empty(valid=False)

random_val = pattern.random()
pattern.flatten(random_val, free=False)

str(pattern)

pattern.empty_bool(True)

# Make sure to test != using a custom test.
testcase.assertTrue(pattern == pattern)

###############################
# Test folding and unfolding.
for free in [True, False, None]:
    for free_default in [True, False, None]:
        pattern.free_default = free_default
        if (free_default is None) and (free is None):
            with testcase.assertRaises(ValueError):
                flat_val = pattern.flatten(valid_value, free=free)
            with testcase.assertRaises(ValueError):
                folded_val = pattern.fold(flat_val, free=free)
        else:
            flat_val = pattern.flatten(valid_value, free=free)
            testcase.assertEqual(len(flat_val), pattern.flat_length(free))
            folded_val = pattern.fold(flat_val, free=free)
            check_equal(valid_value, folded_val)
            if hasattr(valid_value, 'shape'):
                testcase.assertEqual(valid_value.shape, folded_val.shape)



# PSD profiling

In [123]:
import cProfile
import pstats
valid_value = pattern.random()

# Execute required methods.
profile = True

if profile:
    pr = cProfile.Profile()
    pr.enable()

from paragami.psdmatrix_patterns import \
    _vectorize_ld_matrix, _unvectorize_ld_matrix, \
    _pack_posdef_matrix, _unpack_posdef_matrix

mat = np.random.random((30, 30))
mat = mat @ mat + 100 * np.eye(30)

val = _vectorize_ld_matrix(mat)
#jit_fun = _vectorize_ld_matrix
#jit_fun = _unvectorize_ld_matrix
#jit_fun = _pack_posdef_matrix
jit_fun = _unpack_posdef_matrix

tic = time.time()
for _ in range(100):
    jit_fun(val)
    
tic = time.time() - tic; print(tic)

if profile:
    pr.disable()

0.4727363586425781


In [ ]:
ps = pstats.Stats(pr).strip_dirs().sort_stats('cumulative')
ps.print_stats()